In [1]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool, PanTool
from bokeh.models.widgets import Slider, Select, TextInput, Div
from bokeh.models import WheelZoomTool, SaveTool, LassoSelectTool
from bokeh.io import curdoc
from functools import lru_cache

In [13]:
wine_csv = 'https://raw.githubusercontent.com/chris1610/pbpython/master/data/Aussie_Wines_Plotting.csv'

In [45]:
@lru_cache()
def load_data():
    df = pd.read_csv('strava_data.csv', index_col=0)
    return df

In [46]:
df = load_data()

In [48]:
df.head()

,kms,cumulative_daily_actual_kms,daily_goal,cumulative_daily_goal,daily_kms_difference,daily_kms_cumulative_difference,week_number,week,weekly_actual_total,weekly_actual_cumulative,percentage_to_total_weekly,weekly_goal_total,weekly_goal_cumulative,percentage_to_total_weekly_goal,day_of_week,day_number
run_date,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,5.57,5.57,2.86,2.86,1.95,1.95,1,week 1.0,32.62,5.57,0.170754,20.02,2.86,0.142857,Mon,1
2018-01-02 00:00:00+00:00,5.77,11.34,2.86,5.71,2.02,1.99,1,week 1.0,32.62,11.34,0.347639,20.02,5.72,0.285714,Tue,2
2018-01-03 00:00:00+00:00,4.95,16.29,2.86,8.57,1.73,1.90,1,week 1.0,32.62,16.29,0.499387,20.02,8.58,0.428571,Wed,3
2018-01-04 00:00:00+00:00,0.75,17.04,2.86,11.43,0.26,1.49,1,week 1.0,32.62,17.04,0.522379,20.02,11.44,0.571429,Thu,4
2018-01-05 00:00:00+00:00,5.65,22.69,2.86,14.29,1.98,1.59,1,week 1.0,32.62,22.69,0.695586,20.02,14.30,0.714286,Fri,5


In [50]:
# Column order for displaying the details of a specific review
col_order = ["price", "points", "variety", "province", "description"]

all_week = list(set(list(df['week'])))

# all_provinces = [
#     "All", "South Australia", "Victoria", "Western Australia",
#     "Australia Other", "New South Wales", "Tasmania"
# ]

In [51]:
desc = Div(text="Weeks", width=800)
province = Select(title="Week", options=all_week, value="Week_1")
details = Div(text="Selection Details:", width=800)

In [52]:
source = ColumnDataSource(data=load_data())


In [53]:
hover = HoverTool(tooltips=[
    ("Kms", "@kms"),
])
TOOLS = [
    hover, BoxZoomTool(), LassoSelectTool(), WheelZoomTool(), PanTool(),
    ResetTool(), SaveTool()
]

In [23]:
p = figure(
    plot_height=600,
    plot_width=700,
    title="Weekly runs",
    tools=TOOLS,
    x_axis_label="Day of week",
    y_axis_label="KMS",
    toolbar_location="above")

# p.circle(
#     y="price",
#     x="points",
#     source=source,
#     color="variety_color",
#     size=7,
#     alpha=0.4)

p.vbar

GlyphRenderer(id='1061', ...)

In [24]:
def select_reviews():
    """ Use the current selections to determine which filters to apply to the
    data. Return a dataframe of the selected data
    """
    df = load_data()

    # Determine what has been selected for each widgetd
    max_price = price_max.value
    province_val = province.value
    title_val = title.value

    # Filter by price and province
    if province_val == "All":
        selected = df[df.price <= max_price]
    else:
        selected = df[(df.province == province_val) & (df.price <= max_price)]

    # Further filter by string in title if it is provided
    if title_val != "":
        selected = selected[selected.title.str.contains(title_val, case=False) == True]

    # Example showing how to update the description
    desc.text = "Province: {} and Price < {}".format(province_val, max_price)
    return selected

In [25]:
def update():
    """ Get the selected data and update the data in the source
    """
    df_active = select_reviews()
    source.data = ColumnDataSource(data=df_active).data

In [39]:
def selection_change(attrname, old, new):
    """ Function will be called when the poly select (or other selection tool)
    is used. Determine which items are selected and show the details below
    the graph
    """
    selected = source.selected["1d"]["indices"]
    
    df_active = select_reviews()
    
    if selected:
        data = df_active.iloc[selected, :]
        temp = data.set_index("title").T.reindex(index=col_order)
        details.text = temp.style.render()
    else:
        details.text = "Selection Details"
        
        
    

In [40]:
controls = [province, price_max, title]

for control in controls:
    control.on_change("value", lambda attr, old, new: update())

In [41]:
source.on_change("selected", selection_change)

In [42]:
inputs = widgetbox(*controls, sizing_mode="fixed")
l = layout([[desc], [inputs, p], [details]], sizing_mode="fixed")

In [43]:
update()
curdoc().add_root(l)
curdoc().title = "Australian Wine Analysis"